In [1]:
import findspark

findspark.init()

In [2]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd
from pandas import DataFrame
import numpy as np
import matplotlib.pyplot as plt

In [3]:
data_as_pd = pd.read_csv('data/h1b_kaggle.csv')

In [4]:
data_as_pd.head()

,Unnamed: 0,CASE_STATUS,EMPLOYER_NAME,SOC_NAME,JOB_TITLE,FULL_TIME_POSITION,PREVAILING_WAGE,YEAR,WORKSITE,lon,lat
0,1,CERTIFIED-WITHDRAWN,UNIVERSITY OF MICHIGAN,BIOCHEMISTS AND BIOPHYSICISTS,POSTDOCTORAL RESEARCH FELLOW,N,36067.0,2016.0,"ANN ARBOR, MICHIGAN",-83.743038,42.280826
1,2,CERTIFIED-WITHDRAWN,"GOODMAN NETWORKS, INC.",CHIEF EXECUTIVES,CHIEF OPERATING OFFICER,Y,242674.0,2016.0,"PLANO, TEXAS",-96.698886,33.019843
2,3,CERTIFIED-WITHDRAWN,"PORTS AMERICA GROUP, INC.",CHIEF EXECUTIVES,CHIEF PROCESS OFFICER,Y,193066.0,2016.0,"JERSEY CITY, NEW JERSEY",-74.077642,40.728158
3,4,CERTIFIED-WITHDRAWN,"GATES CORPORATION, A WHOLLY-OWNED SUBSIDIARY O...",CHIEF EXECUTIVES,"REGIONAL PRESIDEN, AMERICAS",Y,220314.0,2016.0,"DENVER, COLORADO",-104.990251,39.739236
4,5,WITHDRAWN,PEABODY INVESTMENTS CORP.,CHIEF EXECUTIVES,PRESIDENT MONGOLIA AND INDIA,Y,157518.4,2016.0,"ST. LOUIS, MISSOURI",-90.199404,38.627003


In [5]:
data_as_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3002458 entries, 0 to 3002457
Data columns (total 11 columns):
Unnamed: 0            int64
CASE_STATUS           object
EMPLOYER_NAME         object
SOC_NAME              object
JOB_TITLE             object
FULL_TIME_POSITION    object
PREVAILING_WAGE       float64
YEAR                  float64
WORKSITE              object
lon                   float64
lat                   float64
dtypes: float64(4), int64(1), object(6)
memory usage: 252.0+ MB


In [6]:
# Make a RDD containing only the first 100000 rows
sc = SparkContext.getOrCreate()
sql_sc = SQLContext(sc)

data_rdd = (sql_sc.read.format("csv").options(header="true")
    .load("data/h1b_kaggle.csv").limit(100000))

In [7]:
data_rdd.show()

+---+-------------------+--------------------+--------------------+--------------------+------------------+---------------+----+--------------------+------------+----------+
|_c0|        CASE_STATUS|       EMPLOYER_NAME|            SOC_NAME|           JOB_TITLE|FULL_TIME_POSITION|PREVAILING_WAGE|YEAR|            WORKSITE|         lon|       lat|
+---+-------------------+--------------------+--------------------+--------------------+------------------+---------------+----+--------------------+------------+----------+
|  1|CERTIFIED-WITHDRAWN|UNIVERSITY OF MIC...|BIOCHEMISTS AND B...|POSTDOCTORAL RESE...|                 N|          36067|2016| ANN ARBOR, MICHIGAN| -83.7430378|42.2808256|
|  2|CERTIFIED-WITHDRAWN|GOODMAN NETWORKS,...|    CHIEF EXECUTIVES|CHIEF OPERATING O...|                 Y|         242674|2016|        PLANO, TEXAS| -96.6988856|33.0198431|
|  3|CERTIFIED-WITHDRAWN|PORTS AMERICA GRO...|    CHIEF EXECUTIVES|CHIEF PROCESS OFF...|                 Y|         193066|2016|JE

In [8]:
data_rdd.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- CASE_STATUS: string (nullable = true)
 |-- EMPLOYER_NAME: string (nullable = true)
 |-- SOC_NAME: string (nullable = true)
 |-- JOB_TITLE: string (nullable = true)
 |-- FULL_TIME_POSITION: string (nullable = true)
 |-- PREVAILING_WAGE: string (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- WORKSITE: string (nullable = true)
 |-- lon: string (nullable = true)
 |-- lat: string (nullable = true)



In [9]:
data_rdd.select('CASE_STATUS').show()

+-------------------+
|        CASE_STATUS|
+-------------------+
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|          WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|          WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|CERTIFIED-WITHDRAWN|
|          CERTIFIED|
|          CERTIFIED|
+-------------------+
only showing top 20 rows



In [10]:
data_rdd.select('CASE_STATUS').distinct().count()

4

In [11]:
data_rdd.crosstab('EMPLOYER_NAME', 'CASE_STATUS').show()

+-------------------------+---------+-------------------+------+---------+
|EMPLOYER_NAME_CASE_STATUS|CERTIFIED|CERTIFIED-WITHDRAWN|DENIED|WITHDRAWN|
+-------------------------+---------+-------------------+------+---------+
|      UNIVERSITY OF MAINE|        2|                  0|     0|        0|
|     T.K.CHEN INTERNAT...|        1|                  0|     0|        0|
|        PROFORM GROUP INC|        2|                  0|     0|        0|
|     QUINTUS CONTRACTI...|        1|                  0|     0|        0|
|           SPORTS ACADEMY|        0|                  0|     0|        1|
|        BUENO CONCEPT LLC|        1|                  0|     0|        0|
|     PROGRESS RAIL SER...|        0|                  1|     0|        0|
|          MIROCULUS, INC.|        1|                  0|     0|        0|
|     BLIZZARD ENTERTAI...|        7|                  0|     0|        0|
|     EXPRESS CUSTOMHOU...|        1|                  0|     0|        0|
|            IPOLARITY LL

In [12]:
data_rdd.registerTempTable("data_new")
data_rdd.cache()

#Top10 companies getting visa approval (for year 2016)
sql_sc.sql("SELECT EMPLOYER_NAME, count(EMPLOYER_NAME) as CERTIFIED_COUNT FROM data_new where CASE_STATUS = 'CERTIFIED' AND YEAR='2016' GROUP BY EMPLOYER_NAME order by CERTIFIED_COUNT desc").show(10)

+--------------------+---------------+
|       EMPLOYER_NAME|CERTIFIED_COUNT|
+--------------------+---------------+
|ERNST & YOUNG U.S...|           3394|
|     INFOSYS LIMITED|           2306|
|DELOITTE CONSULTI...|           1707|
|COGNIZANT TECHNOL...|           1390|
|DELOITTE & TOUCHE...|           1131|
|CAPGEMINI AMERICA...|           1105|
|       ACCENTURE LLP|           1018|
|PRICEWATERHOUSECO...|            793|
|            KPMG LLP|            781|
|PRICEWATERHOUSECO...|            700|
+--------------------+---------------+
only showing top 10 rows



In [13]:
#Top10 companies getting visa approval (for year 2016)
sql_sc.sql("SELECT EMPLOYER_NAME, count(EMPLOYER_NAME) as CERTIFIED_COUNT FROM data_new where CASE_STATUS = 'CERTIFIED' AND YEAR='2016' GROUP BY EMPLOYER_NAME order by CERTIFIED_COUNT desc").show(10)

+--------------------+---------------+
|       EMPLOYER_NAME|CERTIFIED_COUNT|
+--------------------+---------------+
|ERNST & YOUNG U.S...|           3394|
|     INFOSYS LIMITED|           2306|
|DELOITTE CONSULTI...|           1707|
|COGNIZANT TECHNOL...|           1390|
|DELOITTE & TOUCHE...|           1131|
|CAPGEMINI AMERICA...|           1105|
|       ACCENTURE LLP|           1018|
|PRICEWATERHOUSECO...|            793|
|            KPMG LLP|            781|
|PRICEWATERHOUSECO...|            700|
+--------------------+---------------+
only showing top 10 rows



In [14]:
#TOP 5 JOB TITLE for which visa are approved in the year 2016
sql_sc.sql("SELECT JOB_TITLE, count(*) as Approved FROM data_new where CASE_STATUS = 'CERTIFIED' AND YEAR='2016' GROUP BY JOB_TITLE order by Approved desc").show(5)

+------------------+--------+
|         JOB_TITLE|Approved|
+------------------+--------+
|  BUSINESS ANALYST|    3664|
|        ACCOUNTANT|    2037|
|PROGRAMMER ANALYST|    1614|
|   SYSTEMS ANALYST|    1375|
| FINANCIAL ANALYST|    1368|
+------------------+--------+
only showing top 5 rows

